In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
# # Membuat list kosong untuk menyimpan data
# judul = []
# tanggal = []
# isi = []
# url_list = []
# kategori_list = []

In [ ]:
# import requests
# from bs4 import BeautifulSoup
# import pandas as pd

# # Fungsi untuk mengambil data dari halaman web Detik.com
# def get_data(url, kategori):
#     # Mengirim permintaan HTTP ke url
#     response = requests.get(url)
#     # Mengecek status kode
#     if response.status_code == 200:
#         # Mengubah konten web menjadi objek BeautifulSoup
#         soup = BeautifulSoup(response.content, "html.parser")
#         # Mencari elemen yang berisi data berita
#         articles = soup.find_all("article", class_="list-content__item")
#         # Melakukan iterasi untuk setiap artikel
#         for article in articles:
#             # Mendapatkan link artikel
#             link = article.find("a")["href"]
#             # Mengirim permintaan HTTP ke link artikel
#             article_response = requests.get(link)
#             # Mengecek status kode
#             if article_response.status_code == 200:
#                 # Mengubah konten artikel menjadi objek BeautifulSoup
#                 article_soup = BeautifulSoup(article_response.content, "html.parser")
#                 # Mendapatkan judul artikel
#                 title_element = article_soup.find("h1", class_="detail__title")
#                 title = title_element.text.strip() if title_element else "Title Not Found"
#                 # Mendapatkan tanggal artikel
#                 date_element = article_soup.find("div", class_="detail__date")
#                 date = date_element.text.strip() if date_element else "Date Not Found"
#                 # Mendapatkan isi artikel
#                 content_element = article_soup.find("div", class_="detail__body-text")
#                 content = content_element.text.strip() if content_element else "Content Not Found"
#                 # Menambahkan data ke list
#                 judul.append(title)
#                 tanggal.append(date)
#                 isi.append(content)
#                 url_list.append(link)
#                 kategori_list.append(kategori)
#                 # Mencetak judul artikel
#                 print(title)
#             else:
#                 # Mencetak pesan error jika status kode tidak 200
#                 print(f"Error: {article_response.status_code}")
#     else:
#         # Mencetak pesan error jika status kode tidak 200
#         print(f"Error: {response.status_code}")

# # Membuat list url dan kategori yang akan di-crawl
# base_urls = ["https://sport.detik.com/indeks", "https://health.detik.com/berita-detikhealth/indeks", "https://travel.detik.com/travel-news/indeks","https://www.detik.com/edu/indeks", "https://oto.detik.com/indeks"," https://food.detik.com/indeks","https://finance.detik.com/indeks"]
# categories = ["Olahraga", "Kesehatan", "Pariwisata", "Pendidikan", "Otomotif",'Kuliner','Keuangan']

# # Inisialisasi list untuk menyimpan data
# judul = []
# tanggal = []
# isi = []
# url_list = []
# kategori_list = []

# # Melakukan iterasi untuk setiap url dan kategori
# for base_url, category in zip(base_urls, categories):
#     # Looping untuk beralih halaman
#     for page in range(0, 50):
#         url = f"{base_url}/{page}"
#         # Memanggil fungsi get_data
#         get_data(url, category)

# # Membuat dataframe dari list data
# df = pd.DataFrame({"judul": judul, "tanggal": tanggal, "isi": isi, "url": url_list, "kategori": kategori_list})

# # Menyimpan dataframe ke file csv
# df.to_csv("data_data_berita_detik.csv", index=False)

# # Menampilkan dataframe
# print(df)

In [ ]:
df = pd.read_csv('/kaggle/input/news-clasification-dataset/data_data_berita_detik (1).csv',encoding="ISO-8859-1")

In [ ]:
df.head()

In [ ]:
import matplotlib.pyplot as plt


# Plotting
plt.figure(figsize=(8, 6))
sns.countplot(data=df, x='kategori', palette='inferno')
plt.title('Jumlah Data Berita per Kategori')

plt.xlabel('Kategori')
plt.ylabel('Jumlah Data')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
feature_nan_count = df.isna().sum().sum()
print(f"Number of NaN values in feature array: {feature_nan_count}")

# Checking for NaN values in labels
label_nan_count = df.isna().sum().sum()
print(f"Number of NaN values in label array: {label_nan_count}")

In [ ]:
import nltk
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import pickle


nltk.download('punkt')
nltk.download('stopwords')


def preprocess_text(text):

    stemmer = PorterStemmer()
    

    words = word_tokenize(text)
    
    
    words = [word.lower() for word in words]
    

    words = [word for word in words if word.isalnum()]
    
  
    words = [word for word in words if not word.isdigit()]

    stop_words = set(stopwords.words('indonesian'))
    
  
    custom_stopwords = ["kondisi","memiliki","indonesia","jakarta","orang","gamba video","simak video","content","video 20detik","video","20detik","simak","gamba","sendiri", "yang", "atau", "dengan", "untuk", "baca", "juga", "advertisement","continue","scroll","dari"]
    stop_words.update(custom_stopwords)
    
    words = [word for word in words if word not in stop_words]
    
    # Perform stemming
    stemmed_words = [stemmer.stem(word) for word in words]
    with open('stemmer.pickle', 'wb') as f:
        pickle.dump(stemmer, f)

    with open('stopwords.pickle', 'wb') as f:
        pickle.dump(stop_words, f)
    
    # Join the words back into a single string
    return ' '.join(stemmed_words)
    # Save stemmer and stop_words as pickle


# Apply the preprocessing function to the 'isi' column
df['isi'] = df['isi'].apply(preprocess_text)

# Print the processed DataFrame
print(df)


In [ ]:
df.head()

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

categories = df['kategori'].unique()
for category in categories:
    # Filter the DataFrame for the current category
    category_df = df[df['kategori'] == category]
    
    # Combine all 'isi' texts for the current category
    combined_text = ' '.join(category_df['isi'])
    
    # Create the word cloud
    wordcloud = WordCloud(width=800, height=400, background_color='black').generate(combined_text)
    
    # Plot the word cloud
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(f'Word Cloud for Category: {category}')
    plt.axis('off')
    plt.show()

In [ ]:
kategori = pd.get_dummies(df.kategori)
df = pd.concat([df, kategori], axis=1)
df = df.drop(columns='kategori')
df.head()

In [ ]:
feature = df['isi']
label = df[['Kesehatan', 'Keuangan', 'Kuliner','Olahraga','Otomotif','Pariwisata','Pendidikan']].values

In [ ]:
print(f"Length of feature: {len(feature)}")
print(f"Length of label: {len(label)}")


In [ ]:

feature_latih, feature_test, label_latih, label_test = train_test_split(feature, label, test_size=0.3, random_state=42)


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


pad_type = 'pre'
trunc_type = 'pre'


tokenizer = Tokenizer(num_words=2000, oov_token='x')
tokenizer.fit_on_texts(feature_latih)
tokenizer.fit_on_texts(feature_test)

sekuens_latih = tokenizer.texts_to_sequences(feature_latih)
sekuens_test = tokenizer.texts_to_sequences(feature_test)


maxlen = max([len(x) for x in sekuens_latih])



padded_latih = pad_sequences(sekuens_latih, padding=pad_type, truncating=trunc_type, maxlen=maxlen)
padded_test = pad_sequences(sekuens_test, padding=pad_type, truncating=trunc_type, maxlen=maxlen)

     

In [ ]:
# # Tokenize the training data
# tokenizer.fit_on_texts(feature_latih)

# # Get the word to index mapping
# word_index = tokenizer.word_index

# # Print the head of the word to index mapping
# print("Word to index mapping (head):")
# for word, index in list(word_index.items())[:5]:
#     print(f"Word: {word}, Index: {index}")


In [ ]:
# # Tokenize the training data
# tokenizer.fit_on_texts(feature_latih)

# # Encode training data sentences into sequences
# sequences_latih = tokenizer.texts_to_sequences(feature_latih)

# # Print some example sequences
# print("Example sequences:")
# for seq in sequences_latih[:1]:  # Print only the first 5 sequences for demonstration
#     print(seq)
#     print(len(seq))
    


In [ ]:
# # Print the head of padded_latih
# print("Head of padded_latih:")
# print(padded_latih[:1].shape)
# print(padded_latih[:2].shape)
# print(padded_latih[:3].shape)


In [ ]:
# Check the type of the input data
print(type(padded_latih))
print(type(label_latih))

Model tersebut adalah model jaringan saraf dalam yang dibangun menggunakan Keras. Berikut adalah penjelasan singkat dari tiap lapisan yang digunakan:

1. **Embedding Layer**: `tf.keras.layers.Embedding(input_dim=2000, output_dim=8)`
   - Layer ini mengubah input berupa integer menjadi representasi vektor berdimensi 8. `input_dim=2000` berarti ada 2000 kata unik yang dapat dimasukkan ke dalam layer ini.

2. **Dropout Layer**: `tf.keras.layers.Dropout(0.0001)`
   - Layer ini digunakan untuk mencegah overfitting dengan cara mengatur proporsi neuron yang dinonaktifkan secara acak selama pelatihan. Di sini, 0.0001 menunjukkan bahwa sangat sedikit neuron yang akan dinonaktifkan.

3. **LSTM Layer**: `tf.keras.layers.LSTM(128)`
   - Layer Long Short-Term Memory (LSTM) dengan 128 unit yang digunakan untuk menangani data urutan atau sekuensial. LSTM berguna untuk mengenali pola dalam urutan data yang panjang.

4. **Dropout Layer**: `tf.keras.layers.Dropout(0.00001)`
   - Sama seperti sebelumnya, tetapi dengan tingkat dropout yang sangat rendah.

5. **Dense Layer**: `tf.keras.layers.Dense(128, activation='sigmoid')`
   - Layer fully connected (Dense) dengan 128 neuron dan fungsi aktivasi sigmoid. Sigmoid digunakan untuk memperkenalkan non-linearitas.

6. **Dropout Layer**: `tf.keras.layers.Dropout(0.00001)`
   - Dropout layer lagi dengan tingkat yang sangat rendah.

7. **Dense Layer**: `tf.keras.layers.Dense(7, activation='sigmoid')`
   - Layer output dengan 7 neuron dan fungsi aktivasi sigmoid, biasanya digunakan untuk menghasilkan prediksi untuk masalah multi-kelas dengan 7 kelas.

Secara keseluruhan, model ini terdiri dari lapisan embedding untuk kata, diikuti oleh LSTM untuk menangkap urutan data, dan beberapa lapisan dense dengan dropout untuk klasifikasi akhir menjadi 7 kelas.

In [ ]:
# model = tf.keras.Sequential([
#   tf.keras.layers.Embedding(input_dim=2000, output_dim=16),
#   tf.keras.layers.Dropout(0.0001),
#   tf.keras.layers.LSTM(128),
#   tf.keras.layers.Dropout(0.00001),
#   tf.keras.layers.Dense(150, activation='sigmoid'),
#   tf.keras.layers.Dropout(0.00001),
#   tf.keras.layers.Dense(7, activation='sigmoid')
#   ])

# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=2000, output_dim=64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)), 
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'), 
    tf.keras.layers.Dropout(0.5),  
    tf.keras.layers.Dense(7, activation='softmax') 
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.9 and logs.get('val_accuracy')>0.84):
      print("\nAkurasi telah mencapai >90%!")
      self.model.stop_training = True
callbacks = myCallback()

In [ ]:

num_epochs = 150
hist = model.fit(padded_latih, label_latih, epochs=num_epochs,
                    validation_data=(padded_test, label_test),
                    verbose=2, callbacks=[callbacks])

In [ ]:
# # Mendapatkan bobot dari lapisan embedding
# embedding_layer = model.get_layer('embedding')
# embedding_weights = embedding_layer.get_weights()[0]

# # Menampilkan embedding
# print("Shape of embedding weights:", embedding_weights.shape)
# print(embedding_weights)

In [ ]:
# # Mendapatkan bobot dari lapisan embedding
# embedding_lstm = model.get_layer('lstm')
# lstm_weights = embedding_lstm.get_weights()[0]

# # Menampilkan embedding
# print("Shape of lstm weights:", embedding_weights.shape)
# print(lstm_weights)

In [ ]:
model.summary()

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Prediksi label dari data uji
label_prediksi = model.predict(padded_test)
label_prediksi = [list(prediksi).index(max(prediksi)) for prediksi in label_prediksi]

# Konversi one-hot encoded labels menjadi label kelas tunggal
label_asli = [list(label).index(1) for label in label_test]

# Buat confusion matrix
cm = confusion_matrix(label_asli, label_prediksi)

# Visualisasi confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Kesehatan', 'Keuangan', 'Kuliner','Olahraga','Otomotif','Pariwisata','Pendidikan'], yticklabels=['Kesehatan', 'Keuangan', 'Kuliner','Olahraga','Otomotif','Pariwisata','Pendidikan'])
plt.title('Confusion Matrix')
plt.show()
     

In [ ]:
# summarize history for accuracy
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
print(maxlen)

# prediksi


In [ ]:
# # Simpan model
# model.save("model_berita_classification1.h5")

# # Simpan tokenizer
# import pickle
# with open('tokenizer1.pickle', 'wb') as handle:
#     pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# # Simpan nilai maxlen
# with open('maxlen.txt', 'w') as f:
#     f.write(str(maxlen))
    


In [ ]:
# import tensorflow as tf
# from tensorflow.keras.models import load_model
# import pickle
# from nltk.corpus import stopwords
# from nltk.stem import PorterStemmer
# from nltk.tokenize import word_tokenize

# # Load stemmer from pickle
# with open('/kaggle/working/stemmer.pickle', 'rb') as f:
#     stemmer = pickle.load(f)

# # Load stopwords from pickle
# with open('/kaggle/working/stopwords.pickle', 'rb') as f:
#     stop_words = pickle.load(f)

# # Now you can define your preprocess_text function with loaded stemmer and stopwords
# def preprocess_text(text):
#     from nltk.tokenize import word_tokenize

#     # Tokenize the text
#     words = word_tokenize(text)

#     # Convert words to lowercase
#     words = [word.lower() for word in words]

#     # Remove punctuation
#     words = [word for word in words if word.isalnum()]

#     # Remove numbers
#     words = [word for word in words if not word.isdigit()]

#     # Remove stopwords
#     words = [word for word in words if word not in stop_words]

#     # Perform stemming using loaded stemmer
#     stemmed_words = [stemmer.stem(word) for word in words]

#     # Join the words back into a single string
#     return ' '.join(stemmed_words)
# # Load tokenizer
# with open('/kaggle/working/tokenizer.pickle', 'rb') as handle:
#     tokenizer = pickle.load(handle)

# # Load model
# model = load_model('/kaggle/working/model_berita_classification.h5')
# # Load maxlen
# with open('maxlen.txt', 'r') as f:
#     maxlen = int(f.read())


# # Contoh data baru yang ingin diprediksi
# data_baru =  ["Irving pernah memperkuat Celtics pada tahun 2017-2019. Selama itu, Irving mampu kemas rata-rata 24 poin per laga dan turut terpilih masuk dalam daftar NBA All-Star.Selepas dari Celtics, Iriving lanjutkan karier ke Brooklyn Nets. Lalu di tahun 2023, pemain berusia 32 tahun itu bergabung ke Dallas Mavericks."]
# preprocessed_data = [preprocess_text(text) for text in data_baru]

# # Preprocessing data baru
# sekuens_data_baru = tokenizer.texts_to_sequences(data_baru)
# padded_data_baru = pad_sequences(sekuens_data_baru, padding=pad_type, truncating=trunc_type, maxlen=maxlen)

# # Prediksi
# hasil_prediksi = model.predict(padded_data_baru)



# # Hasil prediksi
# print(hasil_prediksi)
# categories = ['Kesehatan', 'Keuangan', 'Kuliner', 'Olahraga', 'Otomotif', 'Pariwisata', 'Pendidikan']

# # Mengambil indeks probabilitas tertinggi
# indeks_tertinggi = np.argmax(hasil_prediksi)

# # Mendapatkan kategori sesuai dengan indeks tertinggi
# kategori_prediksi = categories[indeks_tertinggi]

# print("Kategori Prediksi:", kategori_prediksi)



In [ ]:
preprocessed_data

In [ ]:
model.summary()